In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, Astra kekgun example

This is the class method for running Xopt.

In [2]:
from xopt import Xopt

In [3]:
# Make a proper input file. 
YAML="""
xopt: {output_path: temp, verbose: true,
  algorithm: random_sampler}
  
algorithm:
  name: random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10} 

simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator

# Note that you can call another file in the top level group:
vocs: ../templates/kekgun/kekgun_vocs.yaml

"""

In [4]:
!mkdir temp/

In [5]:
# Create object
X = Xopt(config=YAML)

# Change some things to make it run faster
X.vocs['constants']['lspch'] = True
X.vocs['constants']['ipart'] = 1000
X.vocs['constants']['zstop'] = 0.2

# Show config
X

Loading config as text



            Xopt 
________________________________           
Version: 0.4.3
Configured: True
Config as YAML:
xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in,
    workdir: null, astra_bin: /Users/chrisonian/Code/astra/bin/Astra, generator_bin: /Users/chrisonian/Code/astra/bin/generator,
    timeout: 2500, verbose: false, auto_set_spacecharge_mesh: true, archive_path: null,
    merit_f: null}
vocs:
  name: kekgun
  description: Test case
  simulation: astra_with_generator
  templates: {astr

In [6]:
# Check random inputs 
X.random_inputs()

{'sig_x': 0.8112627139540572,
 'lt': 0.040574215009832375,
 'maxe(1)': 46.07430730403743,
 'phi(1)': -3.4895583489880195,
 'maxb(1)': 0.20489241336611724,
 'maxe(2)': 20.570341604498818,
 'phi(2)': 118.27798902745883,
 'maxb(2)': 0.22158153998913704,
 'maxe(3)': 15.200304032023617,
 'maxe(4)': 25.23616457183176,
 'phi(3)': 25.622814087636154,
 'phi(4)': -16.920014300370234,
 'phi(6)': -32.47544828877611,
 'ipart': 1000,
 'lspch': True,
 'zstop': 0.2}

In [7]:
%%time
# Do a random evaluate to check that everything will run
output = X.random_evaluate()
output

CPU times: user 105 ms, sys: 32.6 ms, total: 138 ms
Wall time: 10.2 s


{'error': False,
 'end_mean_z': 0.2,
 'end_norm_emit_x': 1.2313e-06,
 'end_core_emit_95percent_x': 9.594899999999998e-07,
 'end_core_emit_90percent_x': 7.731099999999999e-07,
 'end_core_emit_80percent_x': 5.120400000000001e-07,
 'end_norm_emit_y': 1.2502e-06,
 'end_core_emit_95percent_y': 9.6353e-07,
 'end_core_emit_90percent_y': 7.815e-07,
 'end_core_emit_80percent_y': 5.1746e-07,
 'end_norm_emit_z': 7.4347,
 'end_core_emit_95percent_z': 5.7385e-06,
 'end_core_emit_905percent_z': 4.7812999999999994e-06,
 'end_core_emit_80percent_z': 3.4685e-06,
 'end_mean_t': 7.3598e-10,
 'end_mean_x': 2.8084e-06,
 'end_sigma_x': 0.0016661999999999998,
 'end_sigma_xp': 0.0076426,
 'end_cov_x__xp': 1.273193406e-05,
 'end_mean_y': 2.8924e-06,
 'end_sigma_y': 0.0016661999999999998,
 'end_sigma_yp': 0.007644300000000001,
 'end_cov_y__yp': 1.2734766599999999e-05,
 'end_mean_kinetic_energy': 2197300.0,
 'end_sigma_z': 0.0019687,
 'end_sigma_energy': 32328.000000000004,
 'end_cov_z__energy': -63.209050899999

In [8]:
# These are the algorithm options
X.algorithm['options']

{'verbose': 'T', 'chunk_size': 4, 'max_samples': 10}

In [9]:
# These are the options in the evaluate function
X.simulation['options']

{'astra_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in',
 'generator_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in',
 'workdir': None,
 'astra_bin': '/Users/chrisonian/Code/astra/bin/Astra',
 'generator_bin': '/Users/chrisonian/Code/astra/bin/generator',
 'timeout': 2500,
 'verbose': False,
 'auto_set_spacecharge_mesh': True,
 'archive_path': None,
 'merit_f': None}

# Run Sampler using processes or threads

In [10]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [11]:
# Show config
X.run(executor)

Starting at time 2021-08-10T20:26:39-07:00


███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗██████╗ 
██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔══██╗
███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ██████╔╝
╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ██╔══██╗
███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗██║  ██║
╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚═╝  ╚═╝
                                                           

Xopt Random Sampler
Version 0.4.3

....4 samples completed in 0.28275 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2021-08-10-1.json
....4 samples completed in 0.28853 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2021-08-10-2.json
...

In [12]:
# Check for errors
X.results['error']

[False, False, False, False, False, False, False, False, False, False, False]

# Write this configuration

In [13]:
X.save('test.yaml')

Config written as YAML to test.yaml


In [14]:
!cat test.yaml

xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in,
    workdir: null, astra_bin: /Users/chrisonian/Code/astra/bin/Astra, generator_bin: /Users/chrisonian/Code/astra/bin/generator,
    timeout: 2500, verbose: false, auto_set_spacecharge_mesh: true, archive_path: null,
    merit_f: null}
vocs:
  name: kekgun
  description: Test case
  simulation: astra_with_generator
  templates: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file:

# Run with MPI

In [15]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run test.yaml

  _   
                | |  
__  _____  _ __ | |_ 
\ \/ / _ \| '_ \| __|
 >  < (_) | |_) | |_ 
/_/\_\___/| .__/ \__|
          | |        
          |_|        

_________________________________
Parallel execution with 4 workers
Loading as YAML file: test.yaml

            Xopt 
________________________________           
Version: 0.4.3
Configured: True
Config as YAML:
xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in,
    workdir: null, astra_bin: /Users/chrisonian/Code/astra/bin/Astra, gen

# Dask 

In [16]:
from dask.distributed import Client
executor = Client()
#executor = Client(processes=True)
executor

/Users/chrisonian/Code/miniconda3/envs/devel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57907 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:57907/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:57907/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:57908,Workers: 4
Dashboard: http://127.0.0.1:57907/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:57920,Total threads: 2
Dashboard: http://127.0.0.1:57925/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:57910,


In [17]:
# Wait a few seconds for the Dask cluster to start
from time import sleep
sleep(5)

In [18]:
# Run some more
X.run(executor)

Starting at time 2021-08-10T20:28:34-07:00


███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗██████╗ 
██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔══██╗
███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ██████╔╝
╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ██╔══██╗
███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗██║  ██║
╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚═╝  ╚═╝
                                                           

Xopt Random Sampler
Version 0.4.3

....4 samples completed in 0.24691 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2021-08-10-5.json
....4 samples completed in 0.18002 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2021-08-10-6.json
...

In [19]:
X.results['error']

[False, False, False, False, False, False, False, False, False, False, False]

# Load into Pandas

In [20]:
import pandas as pd
from glob import glob
import json

In [21]:
jfiles = glob('temp/*json')
jfiles

['temp/sampler-2021-08-10-3.json',
 'temp/sampler-2021-08-10-2.json',
 'temp/sampler-2021-08-10-5.json',
 'temp/sampler-2021-08-10-4.json',
 'temp/sampler-2021-08-10-6.json',
 'temp/sampler-2021-08-10-1.json']

In [22]:
# Load a file
data = json.load(open(jfiles[0]))

data['inputs']
data['outputs']
data['error']

[False, False, False, False]

In [23]:
inputs = pd.DataFrame(data['inputs'])
inputs

,sig_x,lt,maxe(1),phi(1),maxb(1),maxe(2),phi(2),maxb(2),maxe(3),maxe(4),phi(3),phi(4),phi(6),ipart,lspch,zstop
0,0.806338,0.059626,21.568042,-4.673121,0.240873,2.897497,-125.043519,0.068095,31.621729,4.721630,-31.134974,28.516834,-40.281337,1000,True,0.2
1,0.826750,0.048359,45.493090,-7.110044,0.336968,0.003778,-134.266925,0.022987,1.100916,2.449849,-27.122572,-36.634193,24.703187,1000,True,0.2
2,0.847150,0.065545,33.218686,8.543800,0.354576,26.467448,-72.438701,0.018615,20.203275,17.254938,34.854429,37.676820,23.768811,1000,True,0.2
3,0.777092,0.048650,27.204583,-23.046581,0.221344,29.441541,-67.009406,0.021003,21.295672,22.701501,14.122687,-16.447998,15.231546,1000,True,0.2


In [24]:
outputs = pd.DataFrame(data['outputs'])
outputs

,error,end_mean_z,end_norm_emit_x,end_core_emit_95percent_x,end_core_emit_90percent_x,end_core_emit_80percent_x,end_norm_emit_y,end_core_emit_95percent_y,end_core_emit_90percent_y,end_core_emit_80percent_y,...,end_sigma_yp,end_cov_y__yp,end_mean_kinetic_energy,end_sigma_z,end_sigma_energy,end_cov_z__energy,end_n_particle_loss,end_total_charge,end_higher_order_energy_spread,fingerprint
0,False,0.2,5.490700e-07,4.230500e-07,3.581600e-07,2.629500e-07,5.459200e-07,4.242900e-07,3.614000e-07,2.645500e-07,...,0.001020,1.038823e-07,1312000.0,0.004175,10281.0,-28.410349,0,9.990000e-11,544.083473,d9689ee5a4ac91013024e373654703e9
1,False,0.2,4.989800e-07,3.981400e-07,3.407700e-07,2.521000e-07,5.040700e-07,3.979700e-07,3.399100e-07,2.543700e-07,...,0.001835,1.391438e-07,2714100.0,0.003887,35211.0,-126.050879,0,9.990000e-11,516.736185,31aa513ade4588ffc848cb3af89e6a2f
2,False,0.2,4.680000e-07,3.852400e-07,3.341300e-07,2.530500e-07,4.711900e-07,3.870900e-07,3.365100e-07,2.549200e-07,...,0.002297,3.118460e-07,1971800.0,0.006016,52224.0,289.773871,0,9.990000e-11,755.254367,94aaecf99c9f9d437ea9f0c33d8338e3
3,False,0.2,5.512800e-07,4.111900e-07,3.490000e-07,2.541700e-07,5.439500e-07,4.142000e-07,3.522600e-07,2.567800e-07,...,0.000797,1.228228e-07,1571700.0,0.002746,40313.0,-109.861493,0,9.990000e-11,1027.676798,58335b174e61bb1388fdc2eaa9d10edf


# Cleanup

In [25]:
!rm -r dask-worker-space

In [26]:
!rm test.yaml

In [27]:
!rm NORRAN

rm: NORRAN: No such file or directory


In [28]:
!rm -r temp